In [144]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.impute import KNNImputer

# Data Cleaning

In [145]:
train = pd.read_csv('train_subset.csv')

In [146]:
test = pd.read_csv('test_subset.csv')

In [147]:
y = pd.read_csv('train_labels.csv')

In [148]:
# merge response + predictors
train = pd.merge(train, y)
test = pd.merge(test, y)

In [149]:
train_filtered = train.drop(['Unnamed: 0','customer_ID','S_2','D_63','D_64'],axis=1)

In [150]:
test_filtered = test.drop(['Unnamed: 0','customer_ID','S_2','D_63','D_64'],axis=1)

In [151]:
# storing missing values for each column in train
val_train = train_filtered.isna().sum()

# storing missing values for each column in test
val_test = test_filtered.isna().sum()

In [152]:
# creating dataframe to store how many values are missing in each column for both train and test
na_df = pd.DataFrame(columns = train_filtered.columns)
test_na_df = pd.DataFrame(columns = test_filtered.columns)
test_na_df.loc['Train'] = val_train
test_na_df.loc['Test'] = val_test
na_df.loc['Train'] = val_train
na_df.loc['Test'] = val_test

In [155]:
# identify predictors with large number of missing values 
col_missing = []
for col in na_df.columns:
    if na_df.loc['Train',col]/10000 > .8:
        col_missing.append(col)

In [156]:
# identify predictors with large number of missing values 
test_col_missing = []
for col in test_na_df.columns:
    if test_na_df.loc['Train',col]/10000 > .8:
        test_col_missing.append(col)

In [157]:
# drop predictors with more than 80% missing values
train_filtered = train_filtered.drop(col_missing,axis=1)

In [160]:
test_filtered = test_filtered.drop(test_col_missing,axis=1)

In [162]:
imputer = KNNImputer(n_neighbors=1)

In [ ]:
train_filtered = pd.DataFrame(imputer.fit_transform(train_filtered),columns = train_filtered.columns)

In [ ]:
test_filtered = pd.DataFrame(imputer.fit_transform(test_filtered),columns = test_filtered.columns)

In [ ]:
train_filtered

In [ ]:
test_filtered

In [128]:
train_filtered.to_csv('train_clean.csv')

In [129]:
test_filtered.to_csv('test_clean.csv')

In [133]:
test_filtered.target

AttributeError: 'DataFrame' object has no attribute 'target'

# Bagging

In [ ]:
# Separating the predictors and response
y = train_filtered.target
X = 